# 6장 : 텍스트와 시퀀스를 위한 딥러닝 *Part 2*

#### 구성
* 텍스트 데이터 다루기
* 순환 신경망 이해하기
* 순환 신경망의 고급 사용법
* 컨브넷을 이용한 시퀀스 처리



#### 이 장에서 다룰 핵심 내용 
* 텍스트 데이터를 유용한 형태로 전처리 하는 방법
* 순환신경망을 사용하는 방법
* 1D 컨브넷을 사용한 데이터 처리

---

## 6.2 순환 신경망 이해하기

완전 연결 네트워크나 컨브넷처럼 지금까지 본 모든 신경망의 특징은 메모리가 없다는 것입니다. 네트워크에 주입되는 입력은 개별적으로 처리되며 입력간에 유지되는 상태가 없습니다. 이런 네트워크로 시퀀스 시계열 데이터 포인트를 처리하려면 네트워크에 전체 시퀀스를 주입해야 합니다. 즉 전체 시퀀스를 하나의 데이터 포인트로 변환해야 합니다. *예를들어 IMDB 문제에서 영화 리뷰를 큰 벡터 하나로 변환하여 처리했스비다. 이런 네트워크를 **피드포워드 네트워크**라고 합니다.* 이와 반대로 사람이 문장을 읽는 겇 처럼 이전에 나온 것을 기억하면서 단어별로 또는 한눈에 들어오는 만큼 처리할 수 있습니다. 이는 문장의 의미를 자연스럽게 표현하도록 도와줍니다. 생물학적 지능은 정보처리를 위한 내부모델을 유지하면서 점진적으로 정보를 처리합니다 이 모델은 과거 정보를 사용하여 구축되며 새롭게 얻은정보를 계속 업데이트합니다.

비록 극단적으로 단순화시킨 버전이지만 **순환신경망 (RNN)** 은 같은원리를 적용한 것 입니다. 시퀀스의 원소를 순회하면서 지금까지 처리한 정보를 상태에 저장합니다. 사실 RNN은 ㅐㄴ부에 루프를 가진 신경망의 한 종류입니다. RNN의 상태는 2개의 다른 시퀀스를 처리하는 사이에 재설정됩니다. (2개의 다른 IMDB리뷰) 하나의 시퀀스가 여전히 하나의 데이터 포인트로 간주됩니다. 즉 네트워크에 하나의 입력을 주입한다고 가정했을 때 이 데이터 포인트가 한번에 처리되지 않는 다는 점이 기존의 네트워크와 다릅니다. 그 대신 네트워크는 시퀀스의 원소를 차례대로 방문합니다.


<center><img src = "img/rnn.png" style = "width : 50%"></center>


In [3]:
import numpy as np

timesteps        = 100
input_features   = 32
output_features  = 64

inputs  = np.random.random((timesteps, input_features))      # 입력 데이터 : 예제를 위해 생성한 난수
state_t = np.zeros((output_features,))                       # 초기 상태   : 모두 0인 벡터

### 랜덤한 가중치 행렬을 만듭니다.
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features, ))

successive_outputs = []
for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    successive_outputs.append(output_t)
    state_t  = output_t

final_output_sequence = np.stack(successive_outputs, axis = 0)

요약하면, RNN은 반복할 때 이전에 계산한 정보를 재사용하는 for문에 지나지않습니다. 물론 이 정의에 맞는 RNN의 종류는 많습니다. RNN은 스텝함수에 의해 특화됩니다.

### 케라스의 순환층


In [4]:
from keras.layers import SimpleRNN

`SimpleRNN`은 두 가지 모드로 실행할 수 있습니다. 각 타임 스텝의 출력을 모은 전체 시퀀스를 반환하거나 (크기가 `(batch_size, timesteps, output_features) `인 3D 텐서) 입력 시퀀스에 대한 마지막 출력만 반환할 수 있습니다. 이 모드에서는 객체를 생성할 떄 `return_sequences` 매개변수로 선택할 수 있습니다. 

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

### 마지막 타임스텝의 출력만 얻는 예제

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [6]:
### 전체 상태의 시퀀스를 반환하는 예제

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences = True))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [7]:
### 네트워크의 표현력을 증가시키기 위해 여러개의 순환층을 차례대로 쌓는 것이 유용할 때가 있습니다. 
### 이런 설정에서는 중간층들이 전체 출력 시퀀스를 반환하도록 설정해야합니다.

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences = True))
model.add(SimpleRNN(32, return_sequences = True))
model.add(SimpleRNN(32, return_sequences = True))
model.add(SimpleRNN(32))         # 맨 위의 층만 마지막 출력을 반환합니다.

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 32)                2080      
Total params: 328,320
Trainable params: 328,320
Non-trainable params: 0
_________________________________________________________________


In [8]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000      # 특성으로 사용할 단어의 수
maxlen       = 500        # 사용할 텍스트의 길이(가장 빈번한 max_features개으 단어만 사용합니다.)
batch_size   = 32

print('loading the data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words = max_features)
print(len(input_train), 'training sequence')
print(len(input_test),  'test sequence')

print('sequence padding (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen = maxlen)
input_test  = sequence.pad_sequences(input_test,  maxlen = maxlen)

print('size of input_train : ', input_train.shape)
print('size of input_test : ',  input_test.shape)


loading the data...
25000 training sequence
25000 test sequence
sequence padding (samples x time)
size of input_train :  (25000, 500)
size of input_test :  (25000, 500)


In [9]:
from keras.layers import Dense

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()
          
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(input_train, y_train, epochs = 10, batch_size = 128, validation_split = 0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 20s 1ms/step - loss: 0.6401 - acc: 0.6175 - val_loss: 0.5105 - val_acc: 0.7702
Epoch 2/10
20000/20000 [==============================] - 20s 982us/step - loss: 0.4176 - acc: 0.8188 - val_loss: 0.3765 - val_acc: 0.8380
Epoch 3/10
20000/20000 [==============================] - 20s 1ms/step - lo

`SimpleRNN`은 텍스트처럼 긴 시퀀스를 처리하는 데 적합하지 않습니다. 더 잘 작동하는 다른 순환 층이 있습니다.

### LSTM과 GRU층 이해하기
케라스에는 `LSTM`과 `GRU`가 있습니다. 실전에서는 항상 이 둘 중에 하나를 사용할 것입니다. `SimpleRNN`은 실전에 쓰기에는 너무 단순하기 때문입니다. `SimpleRNN`은 이론적으로 시간 t에서 이전의 모든 타임스텝의 정보를 유지할 수 있습니다. 실제로는 **긴 시간에 걸친 의존성은 학습할 수 없는 것이 문제** 입니다. 층이 많은 일반 네트워크(피드포워드 네트워크)에서 나타나는 것과 비슷한 현상인 **그레디언트 소실 문제** 때문입니다. 피드포워드 네트워크에 층을 많이 추가할 수록 휸련하기 어려운 것과 같습니다. 

#### LSTM (Long Short-Term Memory)
장, 단기 메모리 알고리즘은 그레디언트 소실문제에 대한 연구의 결정체입니다. *처리할 시퀀스에 나란히 작동하는 컨베이어 벨트를 생각해보세요. 시퀀스 어느 지점에서 추출된 정보가 컨베이어 벨트 위로 올라가 필요한 시점으로 타임스텝으로 이동하여 떨굽니다. 이것이 LSTM이 하는 일입니다.* 나중을 위해 정보를 저장함으로써 처리과정에서 오래된 시그널이 점차 소실되는 것을 막아줍니다.

<center><img src = "img/lstm.png" style = "width : 80%"></center>


In [ ]:
from keras.layers import LSTM
from keras.layers import Embedding
from keras.models import Sequential


model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(input_train, y_train, epochs = 10, batch_size = 128, validation_split = 0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 5376/20000 [=======>......................] - ETA: 1:00 - loss: 0.6605 - acc: 0.6317